In [3]:
%createdb -h fonduer-postgres-dev -U postgres jkracht
#%dropdb -h fonduer-postgres-dev -U postgres jkracht

UsageError: Line magic function `%createdb` not found.


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
import subprocess
import logging
from pprint import pprint

subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'nltk'])

from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')

from fonduer.candidates.matchers import LambdaFunctionMatcher, Union, Intersect, RegexMatchSpan

from fonduer import Meta, init_logging
from fonduer.parser.preprocessors import HTMLDocPreprocessor
from fonduer.parser import Parser
from fonduer.parser.models import Document, Sentence

PARALLEL = 4
ATTRIBUTE = "jkracht"
conn_string = f'postgresql://postgres@fonduer-postgres-dev:5432/{ATTRIBUTE}'

init_logging(log_dir="logs")

session = Meta.init(conn_string).Session()

# Document parser
docs_path = "data/test_collection/"
doc_preprocessor = HTMLDocPreprocessor(docs_path)


print(f"Documents: {session.query(Document).count()}")
print(f"Sentences: {session.query(Sentence).count()}")

[2023-05-15 22:43:53,969][INFO] fonduer.meta:53 - Logging was already initialized to use logs/2023-05-15_22-42-19.  To configure logging manually, call fonduer.init_logging before initialiting Meta.
[2023-05-15 22:43:53,994][INFO] fonduer.meta:135 - Connecting user:postgres to fonduer-postgres-dev:5432/jkracht


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[2023-05-15 22:43:54,448][INFO] fonduer.meta:162 - Initializing the storage schema


In [ ]:
for i in session.query(Document).all():
    print(i.title)

In [9]:
# mention
from fonduer.candidates.models import mention_subclass
from fonduer.candidates import MentionExtractor
from fonduer.candidates import MentionNgrams
from fonduer.candidates.mentions import MentionSentences

Task = mention_subclass("Task")

def mention_span_in_acknowledments_matches_verb(mention):
    # read in span/word
    span_string = mention.get_span() 

    # Task(SpanMention("RefWorks Tagged", sentence=61332, chars=[0,14], words=[0,1]))
    
    for word in span_string.split(): # mention.get_span()[0]
        #print(word)
        try:
            if wn.synsets(word)[0].pos() == "v":
                #print(word)
                return True
        except:
            pass
    return False


    # mention space???
    

matcher_task = LambdaFunctionMatcher(func = mention_span_in_acknowledments_matches_verb)

task_sentences = MentionSentences() ## hier muss noch die Mention-Space eingetragen werden
docs = session.query(Document).all()

mention_extractor = MentionExtractor(
    session,
    [Task],
    [task_sentences],
    [matcher_task],
    parallelism=PARALLEL,
)

mention_extractor.apply(docs, parallelism=PARALLEL, clear=True)
print(
    f"Number of Tasks: {session.query(Task).count()}",
)
